# Light Sources

The `rainbowconnection` allows us to create different light sources. These sources can have very different spectra, but all represent spherically symmetrical emission. Here, we walk through some of the basic properties and functionality of these light sources.

In [ ]:
from rainbowconnection import Sun, Thermal, Incandescent, Sodium
import matplotlib.pyplot as plt, numpy as np
import astropy.units as u

## Basics of `Spectrum` Objects

Let's start by creating a `Spectrum` object to represent the Sun, and then walk through a few of its basic methods.

In [ ]:
s = Sun()
s

The `.plot()` method makes a simple summary plot of the spectrum. By default, it adds a reference rainbow along the top of the plot, and sets the color of the line to match the estimated visual color of the source.

In [ ]:
s.plot();

The `.wavelength()` and `.spectrum()` methods return the spectrum on the wavelength grid on which they have been originally defined. Both have `astropy.units` attached to them; these [units](http://docs.astropy.org/en/stable/units/) ensure consistency amongst all the physical quantities we're using.

In [ ]:
s.wavelength()

In [ ]:
s.spectrum()

The `.integrate` method will integrate the spectrum over specified wavelength limits. If no wavelengths are specified, the integral will be performed over the whole wavelength range.

In [ ]:
s.integrate(400*u.nm, 600*u.nm)

In [ ]:
s.integrate()

## Different Light Sources 

`Sun` is just one example of a `Spectrum` object. We can create lots of different kinds of these objects to represent different kinds of light sources, and all of them will inherit the above methods. Here are a few examples of available sources.

In [ ]:
t = Thermal(teff=7000*u.K, radius=1*u.mm)
t

In [ ]:
i = Incandescent()
i

In [ ]:
na = Sodium(power=12*u.W)
na

We can use the `.at()` method to normalize light sources to be viewed at a particular distance. Most light sources start off expressed in units of luminosity ($W$), but sources viewed `.at` some distance will have units of flux ($W/m^2$).

In [ ]:
solarconstant = Sun().at(1*u.AU)
solarconstant.integrate().to('W/m**2')

## Plotting Light Sources
We can plot or visualize `Spectrum` objects in different ways. We've already seen the `.plot()` method, which also accepts various keyword arguments. The plots are generate with standard `matplotlib.pyplot` tools, so common commands can be used to modify plots that have been created.

In [ ]:
s.plot();

In [ ]:
s.plot(rainbow=False, color='aquamarine', linewidth=3);

To include multiple spectra on the same plot, we can catch the `axes` object returned by each plotting command and feed it in as a keyword argument to future plots. For example, we might want to directly compare the Sun's spectrum to a Planck approximation:

In [ ]:
t = Thermal(5800*u.K, 1*u.Rsun)
ax = t.plot(wavelength=s.wavelength(), color='gray', linestyle='--')
s.plot(ax=ax, color='hotpink')
plt.xscale('log'); plt.yscale('log');

Or, we might want to compare the Planck thermal emission spectra of sources with different temperatures.

In [ ]:
ax = None
for T in np.arange(1000, 20000, 500)*u.K:
    t = Thermal(T)
    ax = t.plot(ax)
plt.yscale('log')
plt.xscale('log')
plt.ylim(1e12, 1e26);

We can plot the spectrum with a rainbow included, to directly visualize the amount of visible light of particular colors.

In [ ]:
s.plot_as_rainbow();

Or we can plot as the spectrum that would be seen visually through a slit spectroscope.

In [ ]:
s.plot_as_slit_rainbow();

We can also integrate the spectrum into (approximate) RGB bins.

In [ ]:
s.plot_rgb();

To access the RGB color of a source, for plotting purposes, use the `.to_color()` method.

In [ ]:
s.to_color()

In [ ]:
# plot the Stefan-Boltzman Law
temperatures = np.arange(1000, 15000, 500)*u.K
plt.plot(temperatures, 5.67e-8*temperatures**4, 
         zorder=-1, color='gray')

# plot individual fluxes, with colors
for T in temperatures:
    t = Thermal(T, radius=1*u.mm).at(1*u.mm)
    plt.scatter(t.teff, t.integrate(), color=t.to_color(), edgecolor='black', s=100)

# tidy up the plot
plt.xlabel('Temperature (K)')
plt.ylabel('Surface Flux $(W/m^2)$');

## Flexible Wavelengths

All `Spectrum` objects have a default grid of internally-defined wavelengths. However, you might want to know the spectrum on your own grid of wavelengths. Most methods that return or do anything related to a spectrum accept a `wavelength=` keyword argument, which you can use to resample onto your desired grid.

In [ ]:
w = np.linspace(.3, .8, 30)*u.um
s.plot(wavelength=w);

In [ ]:
s.spectrum(w)

When resampling to a new wavelength grid, the code will do its best to conserve integrated flux. That is, the integral between two wavelengths should be more-or-less constant across different choices of wavelength grid. 